In [2]:
import os
import time
import pandas as pd

In [25]:



class DataProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None  # Initialize DataFrame to None
        #Initializing class attribute
        self.matched_count = 0
        self.unmatched_count = 0
        self.total_sum = 0
        self.matched_df = None
        self.unmatched_df = None
    
    def create_df(self):
        self.df = pd.read_csv(self.file_path, low_memory=False)
        
    def confirm_balance(self):
        self.create_df()
        self.df['TXN_AMT'] = self.df['TXN_AMT'].str.replace(',', '').astype(float)
        self.total_amount = self.df['TXN_AMT'].sum()
        self.total_amt_count = self.df['TXN_AMT'].count()
        self.total_id_count = self.df['EXTERNAL_REF_NO'].count()
        print(f'"{self.file_path}", {self.total_amount}, {self.total_amt_count}, {self.total_id_count}')
    
    def match_data(self):
        self.create_df()
        self.df['group_key'] = self.df['EXTERNAL_REF_NO'].str.extract('(?<=/)(.*?)(?=_)', expand=False)
        # self.df['TXN_AMT'] = pd.to_numeric(self.df['TXN_AMT'], errors='coerce')
        try:
            self.df['TXN_AMT'] = self.df['TXN_AMT'].str.replace(',', '').astype(float)
        except Exception as e:
            pass
        grouped_df = self.df.groupby('group_key')[['TXN_AMT']].agg(sum=pd.NamedAgg(column='TXN_AMT', aggfunc='sum')).reset_index()
        grouped_df['matched'] = (grouped_df['sum'] == 0).astype(int)
        
        self.matched_df = self.df[self.df['group_key'].isin(grouped_df[grouped_df['matched'] == 1]['group_key'])]
        
        self.unmatched_df = self.df[~self.df['group_key'].isin(grouped_df[grouped_df['matched'] == 1]['group_key'])]
         
        self.total_sum += self.df['TXN_AMT'].sum()
        
        self.unmatched_count += self.unmatched_df['TXN_AMT'].count() #increase the number of unmatched items
        
        self.matched_count += self.matched_df['TXN_AMT'].count() #increase the number of matched items
        
        print(f"Done Matching {self.file_path}: Found {self.matched_df['TXN_AMT'].count()} matched items and {self.unmatched_df['TXN_AMT'].count()} unmatched items...")
        
    def update_record(self):
         return self.total_sum, self.unmatched_count, self.matched_count   
        
    def save_output(self):
        print("Saving output to file...")
        matched_df.to_csv(f"{str(self.file_path)[-17:-4]}_matched.csv", index=False)
        unmatched_df.to_csv(f"{str(self.file_path)[-17:-4]}_unmatched.csv", index=False)
        print("done saving... exiting logic")

        
def split_n_save(df, output_prefix):

    # Calculate how many chunks are needed and adjust last chunk size
    num_chunks = int(df.shape[0] // 1000000)
    last_chunk_size = df.shape[0] % 1000000

    # Get column names (assuming you have them)
    column_names = df.columns

    # Split the dataframe into chunks and save them
    for i in range(num_chunks):
        start_index = i * 1000000
        end_index = (i + 1) * 1000000

        # Adjust ending index for the last chunk
        if i == num_chunks - 1:
            end_index += last_chunk_size

        chunk = df.iloc[start_index:end_index]
        chunk.to_csv(f"{output_prefix}{i+1}.csv", index=False, columns=column_names)



In [19]:
total_matched_count = 0
total_unmatched_count = 0
total_sum = 0


folder_path = r"C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo"
file_list = os.listdir(folder_path)
files = os.path.join(folder_path)

start_time = time.time()
matched_items = pd.DataFrame() #empty df obj for all matched items
unmatched_items = pd.DataFrame() #empty df obj for all unmatched items


# print('"FILE_PATH", "SUM", "AMOUNT COUNT","UNIQUE_ID_COUNT"')
for item in file_list:
    processor = DataProcessor(os.path.join(folder_path, item))
    # processor.confirm_balance()
    processor.match_data()
    matched_items = pd.concat([matched_items, processor.matched_df], ignore_index=True)
    total_matched_count += processor.matched_count
    total_unmatched_count += processor.unmatched_count
    unmatched_items = pd.concat([unmatched_items, processor.unmatched_df], ignore_index=True)
    total_sum += processor.total_sum

    
print(f"Duration of script: {time.time()-start_time} seconds")



"FILE_PATH", "SUM", "AMOUNT COUNT","UNIQUE_ID_COUNT"
Done Matching C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo\spool1_sheet10.csv: Found 933425 matched items and 66575 unmatched items...
Done Matching C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo\spool1_sheet11.csv: Found 899016 matched items and 100984 unmatched items...
Done Matching C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo\spool1_sheet12.csv: Found 970494 matched items and 29506 unmatched items...
Done Matching C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo\spool1_sheet13.csv: Found 949908 matched items and 50092 unmatched items...
Done Matching C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo\spool1_sheet14.csv: Found 927225 matched items and 72775 unmatched items...
Done Matching C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo\spool1_sheet15.csv: Found 959064 matched items and 40936 unmatched items...
Done Matching C:\Users\waliu\Documents\aaaaaaAutom8\spools\combo\spool1_sheet16.csv: Found 981228 ma